# 데이터 불러오기

- 2022년 주유소들의 일자별 가격 데이터
- https://www.opinet.co.kr/user/opdown/opDownload.do

In [1]:
!uv pip install matplotlib

Resolved 11 packages in 157ms
Installed 8 packages in 887ms
 + contourpy==1.3.3
 + cycler==0.12.1
 + fonttools==4.60.1
 + kiwisolver==1.4.9
 + matplotlib==3.10.7
 + numpy==2.3.4
 + pillow==12.0.0
 + pyparsing==3.2.5


In [5]:
# !uv pip install pandas
!uv pip install ipykernel pandas matplotlib

Audited 3 packages in 17ms


In [ ]:
# 상반기, 하반기 주유소 가격 데이터 조회
import pandas as pd

a = pd.read_csv('data/2022년_서울_상반기_일별_가격.csv', encoding='cp949')
a.info
b = pd.read_csv('data/2022년_서울_하반기_일별_가격.csv', encoding='cp949')
b.info


<bound method DataFrame.info of              번호      지역               상호                       주소        기간  \
0      A0006039  서울 강남구  (유)동하석유 힐탑셀프주유소           서울 강남구 논현로 640  20220701   
1      A0006039  서울 강남구  (유)동하석유 힐탑셀프주유소           서울 강남구 논현로 640  20220702   
2      A0006039  서울 강남구  (유)동하석유 힐탑셀프주유소           서울 강남구 논현로 640  20220703   
3      A0006039  서울 강남구  (유)동하석유 힐탑셀프주유소           서울 강남구 논현로 640  20220704   
4      A0006039  서울 강남구  (유)동하석유 힐탑셀프주유소           서울 강남구 논현로 640  20220705   
...         ...     ...              ...                      ...       ...   
81788  A0009138  서울 중랑구            우림주유소  서울 중랑구 용마산로 487 (망우제3동)  20221227   
81789  A0009138  서울 중랑구            우림주유소  서울 중랑구 용마산로 487 (망우제3동)  20221228   
81790  A0009138  서울 중랑구            우림주유소  서울 중랑구 용마산로 487 (망우제3동)  20221229   
81791  A0009138  서울 중랑구            우림주유소  서울 중랑구 용마산로 487 (망우제3동)  20221230   
81792  A0009138  서울 중랑구            우림주유소  서울 중랑구 용마산로 487 (망우제3동)  20221231   

          상표 셀프여부  

In [13]:
# 상반기, 하반기 DataFrame 합치기
# a.info(), b.info()

df = pd.concat([a, b])
df


,번호,지역,상호,주소,기간,상표,셀프여부,고급휘발유,휘발유,경유,실내등유
0,A0006039,서울 강남구,(유)동하석유 힐탑셀프주유소,서울 강남구 논현로 640,20220101,SK에너지,셀프,1887,1737,1587,0
1,A0006039,서울 강남구,(유)동하석유 힐탑셀프주유소,서울 강남구 논현로 640,20220102,SK에너지,셀프,1887,1737,1587,0
2,A0006039,서울 강남구,(유)동하석유 힐탑셀프주유소,서울 강남구 논현로 640,20220103,SK에너지,셀프,1887,1737,1587,0
3,A0006039,서울 강남구,(유)동하석유 힐탑셀프주유소,서울 강남구 논현로 640,20220104,SK에너지,셀프,1887,1737,1587,0
4,A0006039,서울 강남구,(유)동하석유 힐탑셀프주유소,서울 강남구 논현로 640,20220105,SK에너지,셀프,1887,1737,1587,0
...,...,...,...,...,...,...,...,...,...,...,...
81788,A0009138,서울 중랑구,우림주유소,서울 중랑구 용마산로 487 (망우제3동),20221227,GS칼텍스,셀프,0,1525,1745,0
81789,A0009138,서울 중랑구,우림주유소,서울 중랑구 용마산로 487 (망우제3동),20221228,GS칼텍스,셀프,0,1525,1720,0
81790,A0009138,서울 중랑구,우림주유소,서울 중랑구 용마산로 487 (망우제3동),20221229,GS칼텍스,셀프,0,1525,1720,0
81791,A0009138,서울 중랑구,우림주유소,서울 중랑구 용마산로 487 (망우제3동),20221230,GS칼텍스,셀프,0,1515,1715,0


In [14]:
# DataFrame 기본 정보 조회
df.shape
df.info()


<class 'pandas.core.frame.DataFrame'>
Index: 164482 entries, 0 to 81792
Data columns (total 11 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   번호      164482 non-null  object
 1   지역      164482 non-null  object
 2   상호      164482 non-null  object
 3   주소      164482 non-null  object
 4   기간      164482 non-null  int64 
 5   상표      164482 non-null  object
 6   셀프여부    164482 non-null  object
 7   고급휘발유   164482 non-null  int64 
 8   휘발유     164482 non-null  int64 
 9   경유      164482 non-null  int64 
 10  실내등유    164482 non-null  int64 
dtypes: int64(5), object(6)
memory usage: 15.1+ MB


In [15]:
# 상표 컬럼의 고유값 조회
unique_brands = df['상표'].unique()

print(unique_brands)

['SK에너지' 'GS칼텍스' '현대오일뱅크' 'S-OIL' '알뜰주유소' '자가상표' '알뜰(ex)']


In [ ]:
# 지역 컬럼의 고유값 조회
unique_region = df['지역'].unique()

print(unique_region)


['서울 강남구' '서울 강동구' '서울 강북구' '서울 강서구' '서울 관악구' '서울 광진구' '서울 구로구' '서울 금천구'
 '서울 노원구' '서울 도봉구' '서울 동대문구' '서울 동작구' '서울 마포구' '서울 서대문구' '서울 서초구' '서울 성동구'
 '서울 성북구' '서울 송파구' '서울 양천구' '서울 영등포구' '서울 용산구' '서울 은평구' '서울 종로구' '서울 중구'
 '서울 중랑구']


> ### str accessor
> - `Series.str.String메소드()`
> - 문자열 타입 Series에 String 메소드들을 사용하여 일괄처리 할 수 있도록 한다.
> - str accessor 메소드 : https://pandas.pydata.org/pandas-docs/stable/reference/series.html#api-series-str

> ### datetime으로 변환
> - `pd.to_datetime(변환할 Series, format="날짜형식")`
> - x 또는 y 축이 날짜 일 경우 datetime 타입으로 처리하는 것이 좋다.
> - format: 년월일의 형식을 strftime 형식문자열로 지정한다. 
>      - [strftime 형식](https://docs.python.org/3/library/datetime.html#strftime-and-strptime-format-codes)  
> - datetime을 변환하면 날짜만의 기능을 다룰 수 있다.
>     - dt accessor를 이용해 datetime의 속성값들을(년, 월, 일 등) 추출할 수 있다.
>     - https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html


In [20]:
# 지역 컬럼의 값을 이용하여 "구" 컬럼 생성 (지역에서 구만 추출해서 파생변수컬럼 생성해서 구별로로 집계를 할 수있도록 한다.)
unique_region = df['지역'].unique()
s = pd.Series(unique_region)
gu_list = s.str.split(' ').str[1]
gu_list

0      강남구
1      강동구
2      강북구
3      강서구
4      관악구
5      광진구
6      구로구
7      금천구
8      노원구
9      도봉구
10    동대문구
11     동작구
12     마포구
13    서대문구
14     서초구
15     성동구
16     성북구
17     송파구
18     양천구
19    영등포구
20     용산구
21     은평구
22     종로구
23      중구
24     중랑구
dtype: object

In [26]:
# 기간 컬럼을 datetime 타입으로 변경한다.
date_span = df['기간']
date_span
s = pd.Series(date_span)
s_datetime = pd.to_datetime(s, format='%Y%m%d')
s_datetime



0       2022-01-01
1       2022-01-02
2       2022-01-03
3       2022-01-04
4       2022-01-05
           ...    
81788   2022-12-27
81789   2022-12-28
81790   2022-12-29
81791   2022-12-30
81792   2022-12-31
Name: 기간, Length: 164482, dtype: datetime64[ns]

In [33]:
# 기간 컬럼을 이용해 "월", "일", "요일" 컬럼 생성 (dt accessor 이용) (월, 일, 요일 별로 집계할 수있도록 한다.)
df2 = df.copy()
df2['기간']
df2['기간2'] = pd.to_datetime(df2['기간'], format='%Y%m%d')
df2['월'] = df2['기간2'].dt.month
df2['일'] = df2['기간2'].dt.day
df2['요일'] = df2['기간2'].dt.dayofweek
df2.tail(100)

,번호,지역,상호,주소,기간,상표,셀프여부,고급휘발유,휘발유,경유,실내등유,기간2,월,일,요일
81693,A0009138,서울 중랑구,우림주유소,서울 중랑구 용마산로 487 (망우제3동),20220923,GS칼텍스,셀프,0,1675,1815,0,2022-09-23,9,23,4
81694,A0009138,서울 중랑구,우림주유소,서울 중랑구 용마산로 487 (망우제3동),20220924,GS칼텍스,셀프,0,1675,1815,0,2022-09-24,9,24,5
81695,A0009138,서울 중랑구,우림주유소,서울 중랑구 용마산로 487 (망우제3동),20220925,GS칼텍스,셀프,0,1675,1815,0,2022-09-25,9,25,6
81696,A0009138,서울 중랑구,우림주유소,서울 중랑구 용마산로 487 (망우제3동),20220926,GS칼텍스,셀프,0,1665,1797,0,2022-09-26,9,26,0
81697,A0009138,서울 중랑구,우림주유소,서울 중랑구 용마산로 487 (망우제3동),20220927,GS칼텍스,셀프,0,1665,1797,0,2022-09-27,9,27,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81788,A0009138,서울 중랑구,우림주유소,서울 중랑구 용마산로 487 (망우제3동),20221227,GS칼텍스,셀프,0,1525,1745,0,2022-12-27,12,27,1
81789,A0009138,서울 중랑구,우림주유소,서울 중랑구 용마산로 487 (망우제3동),20221228,GS칼텍스,셀프,0,1525,1720,0,2022-12-28,12,28,2
81790,A0009138,서울 중랑구,우림주유소,서울 중랑구 용마산로 487 (망우제3동),20221229,GS칼텍스,셀프,0,1525,1720,0,2022-12-29,12,29,3
81791,A0009138,서울 중랑구,우림주유소,서울 중랑구 용마산로 487 (망우제3동),20221230,GS칼텍스,셀프,0,1515,1715,0,2022-12-30,12,30,4


In [ ]:
# 요일 - 0:월, 1:화, 2:수, 3:목, 4:금, 5:토, 6:일 로 변경


In [ ]:
# 휘발유 가격이 가장 비싼 5개 행 조회
# df2['위발유'].max().head(5)
df_sorted = df.sort_values(by='휘발유', ascending=False)
top_5_expensive = df_sorted.head(5)
top_5_expensive


,번호,지역,상호,주소,기간,상표,셀프여부,고급휘발유,휘발유,경유,실내등유
78938,A0000767,서울 중구,서남주유소,서울 중구 통일로 30,20220625,SK에너지,일반,3496,3096,3223,2719
78935,A0000767,서울 중구,서남주유소,서울 중구 통일로 30,20220622,SK에너지,일반,3496,3096,3223,2719
78940,A0000767,서울 중구,서남주유소,서울 중구 통일로 30,20220627,SK에너지,일반,3496,3096,3223,2719
78936,A0000767,서울 중구,서남주유소,서울 중구 통일로 30,20220623,SK에너지,일반,3496,3096,3223,2719
78939,A0000767,서울 중구,서남주유소,서울 중구 통일로 30,20220626,SK에너지,일반,3496,3096,3223,2719
78937,A0000767,서울 중구,서남주유소,서울 중구 통일로 30,20220624,SK에너지,일반,3496,3096,3223,2719
78943,A0000767,서울 중구,서남주유소,서울 중구 통일로 30,20220630,SK에너지,일반,3481,3081,3230,2703
77931,A0000767,서울 중구,서남주유소,서울 중구 통일로 30,20220703,SK에너지,일반,3481,3081,3230,2703
78942,A0000767,서울 중구,서남주유소,서울 중구 통일로 30,20220629,SK에너지,일반,3481,3081,3230,2703
78941,A0000767,서울 중구,서남주유소,서울 중구 통일로 30,20220628,SK에너지,일반,3481,3081,3230,2703


In [ ]:
# 휘발유 가격이 가장 저렴한 5개 행 조회 (단 0원인 주유소 제외)


In [ ]:
# 고급휘발유 가격이 가장 비싼 5개 행 조회


In [ ]:
# 고급휘발유 가격이 가장 저렴한 5개 행 조회 (단 0원인 주유소 제외)


In [ ]:
# 경유 가격이 가장 비싼 5개 행 조회


In [ ]:
# 경유 가격이 가장 저렴한 5개 행 조회 (단 0원인 주유소 제외)


In [ ]:
# 상표별 휘발유 평균가격 조회


In [ ]:
# 상표별 휘발유 평균가격 시각화


In [ ]:
# 셀프주유소와 일반주유소 개수 확인


In [ ]:
# 셀프 여부에 따른 휘발유 평균 가격 비교 


In [ ]:
# 상표별 일반/셀프 주유소의 휘발유 가격 평균 비교


In [ ]:
# 상표별 일반/셀프 주유소의 휘발유 가격 평균 비교 시각화


In [ ]:
# 상표별 휘발유 가격의 분포를 boxplot으로 시각화


In [ ]:
# 상표별 일반/셀프 주유소의 휘발유 가격에 대한 분포 확인 - boxplot으로 시각화


In [ ]:
# 구별 휘발유 가격의 평균


In [ ]:
# 평균 가격이 가장 높은 구의 상표별 휘발유 평균가격


In [ ]:
# 월 평균 휘발유 가격


In [ ]:
# 각 구의 월별 평균 휘발유 가격 


In [ ]:
# 각 구의 월별 평균가격의 변화추이 시각화


In [ ]:
# 가장 휘발유가격이 비싼 주유소 상위 5 - (같은 주유소가 여러개 나오므로 가장 비싼 가격을 기준으로 집계)


In [ ]:
# 가장 휘발유가격이 저렴한 주유소 상위 5 - (같은 주유소가 여러개 나오므로 가장 비싼 가격을 기준으로 집계)


In [ ]:
# 휘발유 비싼 주유소의 상위 5개의 상호, 주소, 상표, 셀프여부,  조회 - 위에서 조회한 결과에 추가 정보(상표, 셀프여부 등) 조회 (중복제거-drop_duplicates)


In [ ]:
# 휘발유 저렴한 주유소의 상위 5개의 상호, 주소, 상표, 셀프여부,  조회 - 위에서 조회한 결과에 추가 정보(상표, 셀프여부 등) 조회 (중복제거-drop_duplicates)


In [ ]:
# 고급 휘발유를 가장 많이 파는 구 조회


In [ ]:
###############################################################################
# 주유소 가격 데이터를 이용해서 알고 싶은 정보들을 추가 분석해 보세요.
###############################################################################
